In [1]:
import syft as sy
sy.logger.remove()
import numpy as np
import pandas as pd
data = sy.Tensor(np.array([1,2,3],dtype=np.int32))
from syft.core.adp.entity import DataSubject
from syft.core.tensor.smpc.mpc_tensor import MPCTensor
import time
import uuid
from typing import Any
from typing import Dict

In [2]:

p1 =  sy.login(email="info@openmined.org",password="changethis",port="9082")
p2 = sy.login(email="info@openmined.org",password="changethis",port="9083")

Connecting to http://localhost:9082... done! 	 Logging into ninja... done!
Connecting to http://localhost:9083... done! 	 Logging into musing_wolf... done!


In [3]:
ROOT_DIR = "/home/rasswanth/PySyft"
def load_data(csv_file: str) -> pd.DataFrame:
    return pd.read_csv(f"{ROOT_DIR}/notebooks/trade_demo/datasets/{csv_file}")[0:10]

def get_user_details(unique_email: str) -> Dict[str, Any]:
    return {
        "name": "Sheldon Cooper",
        "email": unique_email,
        "password": "bazinga",
        "budget": 10,
    }

unique_email = f"{uuid.uuid4()}@caltech.edu"

# Canada
ca_root = sy.login(email="info@openmined.org", password="changethis", port=9082)
ca_data = load_data(csv_file="ca - feb 2021.csv")

# NOTE: casting this tensor as np.int32 is REALLY IMPORTANT
canada_trade = (
    (np.array(list(ca_data["Trade Value (US$)"])) / 100000)[0:10]
).astype(np.int32)
trade_partners = ((list(ca_data["Partner"])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(DataSubject(name=trade_partners[i]))

sampled_canada_dataset = sy.Tensor(canada_trade)
sampled_canada_dataset.public_shape = sampled_canada_dataset.shape
sampled_canada_dataset = sampled_canada_dataset.private(
    0, 3, entities=entities[0]
).tag("trade_flow")

# load dataset
ca_root.load_dataset(
    assets={"Canada Trade": sampled_canada_dataset},
    name="Canada Trade Data - First few rows",
    description=(
        "A collection of reports from Canada's statistics bureau about how "
        + "much it thinks it imports and exports from other countries."
    ),
    skip_checks=True,
)

assert len(ca_root.datasets) > 0



# Italy
it_root = sy.login(email="info@openmined.org", password="changethis", port=9083)
it_data = load_data(csv_file="it - feb 2021.csv")
# NOTE: casting this tensor as np.int32 is REALLY IMPORTANT
data_batch = ((np.array(list(it_data["Trade Value (US$)"])) / 100000)[0:10]).astype(
    np.int32
)
trade_partners = ((list(it_data["Partner"])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(DataSubject(name="Other Asia, nes"))

# Upload a private dataset to the Domain object, as the root owner
sampled_italy_dataset = sy.Tensor(data_batch)
sampled_italy_dataset.public_shape = sampled_italy_dataset.shape
sampled_italy_dataset = sampled_italy_dataset.private(
    0, 3, entities=entities[0]
).tag("trade_flow")

it_root.load_dataset(
    assets={"Italy Trade": sampled_italy_dataset},
    name="Italy Trade Data - First few rows",
    description=(
        "A collection of reports from iStat's statistics bureau about how "
        + "much it thinks it imports and exports from other countries."
    ),
    skip_checks=True,
)



Connecting to http://localhost:9082... done! 	 Logging into ninja... done!


/tmp/ipykernel_25044/2682176243.py:17: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  ca_data = load_data(csv_file="ca - feb 2021.csv")


Loading dataset... uploading... SUCCESS!                                                                                                                                      

Run <your client variable>.datasets to see your new dataset loaded into your machine!
Connecting to http://localhost:9083... done! 	 Logging into musing_wolf... done!
Loading dataset... uploading... SUCCESS!                                                                                                                                      

Run <your client variable>.datasets to see your new dataset loaded into your machine!


/tmp/ipykernel_25044/2682176243.py:52: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  it_data = load_data(csv_file="it - feb 2021.csv")


In [6]:
x = p1.datasets[-1]["Canada Trade"]

In [7]:
y = p2.datasets[-1]["Italy Trade"]

In [13]:
t=x-y

Reeeeemote ring_size 4294967296
/home/rasswanth/PySyft/packages/syft/src/syft/ast/klass.py:__getattribute__:901
Reeeeemote ring_size 4294967296
/home/rasswanth/PySyft/packages/syft/src/syft/ast/klass.py:__getattribute__:901
Reeeeemote ring_size 4294967296
/home/rasswanth/PySyft/packages/syft/src/syft/ast/klass.py:__getattribute__:901
Reeeeemote ring_size 4294967296
/home/rasswanth/PySyft/packages/syft/src/syft/ast/klass.py:__getattribute__:901


In [15]:
t1 = x.get_copy()

In [14]:
t2 = y.get_copy()

In [18]:
t1-t2

Exception: Cannot add two tensors with different symbol encodings

In [11]:
public_result = t.publish(sigma=2)

In [12]:
t.get()

OP <built-in function add> int32 4294967296


Tensor(child=SingleEntityPhiTensor(entity=Other Asia, nes, child=ShareTensor(child=[ 0  2 17  0  1 14  0  4  0  0])))

In [ ]:
time.sleep(5)
sycure_result = public_result.get()
print("sycure_result", sycure_result)
print("after ca", ca.privacy_budget)
print("after it", it.privacy_budget)

assert len(sycure_result) == 10
assert sum(sycure_result) > -100
assert sum(sycure_result) < 100

assert ca.privacy_budget < 200
assert ca.privacy_budget > 10
assert it.privacy_budget < 200
assert it.privacy_budget > 10
assert ca.privacy_budget == it.privacy_budget